In [1]:
import csv
import xml.etree.ElementTree as et
import numpy as np
import pandas as pd
from datetime import datetime as dt

In [2]:
matches = ['f24-4-2013-731767-eventdetails.xml','f24-4-2013-731769-eventdetails.xml','f24-4-2013-731773-eventdetails.xml',
'f24-4-2013-731774-eventdetails.xml','f24-4-2013-731777-eventdetails.xml','f24-4-2013-731779-eventdetails.xml',
'f24-4-2013-731781-eventdetails.xml','f24-4-2013-731783-eventdetails.xml','f24-4-2013-731785-eventdetails.xml',
'f24-4-2013-731786-eventdetails.xml','f24-4-2013-731789-eventdetails.xml','f24-4-2013-731790-eventdetails.xml',
'f24-4-2013-731793-eventdetails.xml','f24-4-2013-731795-eventdetails.xml','f24-4-2013-731798-eventdetails.xml',
'f24-4-2013-731799-eventdetails.xml','f24-4-2013-731801-eventdetails.xml','f24-4-2013-731802-eventdetails.xml',
'f24-4-2013-731806-eventdetails.xml','f24-4-2013-731809-eventdetails.xml','f24-4-2013-731811-eventdetails.xml',
'f24-4-2013-731814-eventdetails.xml','f24-4-2013-731815-eventdetails.xml','f24-4-2013-731817-eventdetails.xml',
'f24-4-2013-731818-eventdetails.xml','f24-4-2013-731820-eventdetails.xml','f24-4-2013-731821-eventdetails.xml',
'f24-4-2013-731822-eventdetails.xml','f24-4-2013-731823-eventdetails.xml','f24-4-2013-731824-eventdetails.xml',
'f24-4-2013-731825-eventdetails.xml','f24-4-2013-731826-eventdetails.xml','f24-4-2013-731827-eventdetails.xml',
'f24-4-2013-731828-eventdetails.xml','f24-4-2013-731829-eventdetails.xml','f24-4-2013-731830-eventdetails.xml']

In [3]:
x_start = []
y_start = []
x_end = []
y_end = []
outcome = []
minute = []
half = []
team = []
game_num = []
type_id = []

In [4]:
for x in matches:
    
    tree = et.ElementTree(file = x)
    gameFile = tree.getroot()
    team_dict = {gameFile[0].attrib["home_team_id"]: gameFile[0].attrib["home_team_name"],
                 gameFile[0].attrib["away_team_id"]: gameFile[0].attrib["away_team_name"]}

    for game in gameFile:

        for event in game:
            
            if event.attrib.get("outcome") == '1':
    
                if event.attrib.get("type_id") == '1':
                
                    x_start.append(event.attrib.get("x"))
                    y_start.append(event.attrib.get("y"))
                    outcome.append('Successful')
                    minute.append(event.attrib.get("min"))
                    half.append(event.attrib.get("period_id"))
                    team.append(team_dict[event.attrib.get("team_id")])
                    game_num.append(x)
                    type_id.append(event.attrib.get("type_id"))
                
                
                    for qualifier in event:
                    
                        if qualifier.attrib.get("qualifier_id") == "140":
                            x_end.append(qualifier.attrib.get("value"))
                        if qualifier.attrib.get("qualifier_id") == "141":
                            y_end.append(qualifier.attrib.get("value"))

In [5]:
for x in matches:
    
    tree = et.ElementTree(file = x)
    gameFile = tree.getroot()
    team_dict = {gameFile[0].attrib["home_team_id"]: gameFile[0].attrib["home_team_name"],
                 gameFile[0].attrib["away_team_id"]: gameFile[0].attrib["away_team_name"]}

    for game in gameFile:

        for event in game:
            
            if event.attrib.get("outcome") == '0':
    
                if event.attrib.get("type_id") == '1':
                
                    x_start.append(event.attrib.get("x"))
                    y_start.append(event.attrib.get("y"))
                    outcome.append('Unsuccessful')
                    minute.append(event.attrib.get("min"))
                    half.append(event.attrib.get("period_id"))
                    team.append(team_dict[event.attrib.get("team_id")])
                    game_num.append(x)
                    type_id.append(event.attrib.get("type_id"))
                
                
                    for qualifier in event:
                    
                        if qualifier.attrib.get("qualifier_id") == "140":
                            x_end.append(qualifier.attrib.get("value"))
                        if qualifier.attrib.get("qualifier_id") == "141":
                            y_end.append(qualifier.attrib.get("value"))

In [6]:
column_titles = ["team", "half", "min", "x_start", "y_start", "x_end", "y_end", "outcome", "game", "type_id"]            
final_table = pd.DataFrame(data=[team, half, minute, x_start, y_start, x_end, y_end, outcome, game_num, type_id],
                           index=column_titles)
final_table = final_table.T
final_table

,team,half,min,x_start,y_start,x_end,y_end,outcome,game,type_id
0,Croatia,1,0,50.1,50.7,53.1,50.0,Successful,f24-4-2013-731767-eventdetails.xml,1
1,Croatia,1,0,53.1,50.0,38.2,52.5,Successful,f24-4-2013-731767-eventdetails.xml,1
2,Croatia,1,0,38.2,53.0,20.9,48.2,Successful,f24-4-2013-731767-eventdetails.xml,1
3,Croatia,1,0,20.9,48.2,9.0,39.1,Successful,f24-4-2013-731767-eventdetails.xml,1
4,Brazil,1,0,68.6,100.0,72.8,85.7,Successful,f24-4-2013-731767-eventdetails.xml,1
...,...,...,...,...,...,...,...,...,...,...
36090,Germany,4,120,5.0,44.8,76.9,0.0,Unsuccessful,f24-4-2013-731830-eventdetails.xml,1
36091,Germany,4,123,5.3,51.2,67.7,37.3,Unsuccessful,f24-4-2013-731830-eventdetails.xml,1
36092,Argentina,4,123,14.6,67.0,67.9,95.6,Unsuccessful,f24-4-2013-731830-eventdetails.xml,1
36093,Germany,4,123,26.1,20.6,42.2,15.0,Unsuccessful,f24-4-2013-731830-eventdetails.xml,1


In [7]:
final_table = final_table.replace('f24-4-2013-731767-eventdetails.xml','Brazil vs Croatia')
final_table = final_table.replace('f24-4-2013-731769-eventdetails.xml','Spain vs Netherlands')
final_table = final_table.replace('f24-4-2013-731773-eventdetails.xml','Uruguay vs Costa Rica')
final_table = final_table.replace('f24-4-2013-731774-eventdetails.xml','England vs Italy')
final_table = final_table.replace('f24-4-2013-731777-eventdetails.xml','Argentina vs Bosnia and Herzegovina')
final_table = final_table.replace('f24-4-2013-731779-eventdetails.xml','Germany vs Portugal')
final_table = final_table.replace('f24-4-2013-731781-eventdetails.xml','Belgium vs Algeria')
final_table = final_table.replace('f24-4-2013-731783-eventdetails.xml','Brazil vs Mexico')
final_table = final_table.replace('f24-4-2013-731785-eventdetails.xml','Spain vs Chile')
final_table = final_table.replace('f24-4-2013-731786-eventdetails.xml','Australia vs Netherlands')
final_table = final_table.replace('f24-4-2013-731789-eventdetails.xml','Uruguay vs England')
final_table = final_table.replace('f24-4-2013-731790-eventdetails.xml','Italy vs Costa Rica')
final_table = final_table.replace('f24-4-2013-731793-eventdetails.xml','Argentina vs Iran')
final_table = final_table.replace('f24-4-2013-731795-eventdetails.xml','Germany vs Ghana')
final_table = final_table.replace('f24-4-2013-731798-eventdetails.xml','South Korea vs Algeria')
final_table = final_table.replace('f24-4-2013-731799-eventdetails.xml','Cameroon vs Brazil')
final_table = final_table.replace('f24-4-2013-731801-eventdetails.xml','Australia vs Spain')
final_table = final_table.replace('f24-4-2013-731802-eventdetails.xml','Netherlands vs Chile')
final_table = final_table.replace('f24-4-2013-731806-eventdetails.xml','Costa Rica vs England')
final_table = final_table.replace('f24-4-2013-731809-eventdetails.xml','Nigeria vs Argentina')
final_table = final_table.replace('f24-4-2013-731811-eventdetails.xml','USA vs Germany')
final_table = final_table.replace('f24-4-2013-731814-eventdetails.xml','Algeria vs Russia')
final_table = final_table.replace('f24-4-2013-731815-eventdetails.xml','Brazil vs Chile')
final_table = final_table.replace('f24-4-2013-731817-eventdetails.xml','Netherlands vs Mexico')
final_table = final_table.replace('f24-4-2013-731818-eventdetails.xml','Costa Rica vs Greece')
final_table = final_table.replace('f24-4-2013-731820-eventdetails.xml','Germany vs Algeria')
final_table = final_table.replace('f24-4-2013-731821-eventdetails.xml','Argentina vs Switzerland')
final_table = final_table.replace('f24-4-2013-731822-eventdetails.xml','Belgium vs USA')
final_table = final_table.replace('f24-4-2013-731823-eventdetails.xml','Brazil vs Colombia')
final_table = final_table.replace('f24-4-2013-731824-eventdetails.xml','France vs Germany')
final_table = final_table.replace('f24-4-2013-731825-eventdetails.xml','Netherlands vs Costa Rica')
final_table = final_table.replace('f24-4-2013-731826-eventdetails.xml','Argentina vs Belgium')
final_table = final_table.replace('f24-4-2013-731827-eventdetails.xml','Brazil vs Germany')
final_table = final_table.replace('f24-4-2013-731828-eventdetails.xml','Netherlands vs Argentina')
final_table = final_table.replace('f24-4-2013-731829-eventdetails.xml','Brazil vs Netherlands')
final_table = final_table.replace('f24-4-2013-731830-eventdetails.xml','Germany vs Argentina')

In [8]:
final_table.to_csv('passes.csv')

In [9]:
df = pd.read_csv('passes.csv')

In [10]:
df['x_start'] = df['x_start']*1.2
df['y_start'] = df['y_start']*.8
df['x_end'] = df['x_end']*1.2
df['y_end'] = df['y_end']*.8

In [11]:
df['beginning'] = np.sqrt(np.square(120-df['x_start'])+np.square(40-df['y_start']))
df['end'] = np.sqrt(np.square(120-df['x_end'])+np.square(40-df['y_end']))
df['progressive'] = [(df['end'][x]) / (df['beginning'][x]) < .75 for x in range(len(df.beginning))]

In [12]:
xT = pd.read_csv('xT_grid.csv',header=None)
xT = np.array(xT)
xT_rows, xT_cols = xT.shape
df['x1_bin'] = pd.cut(df['x_start'], bins = xT_cols, labels=False)
df['y1_bin'] = pd.cut(df['y_start'], bins = xT_rows, labels=False)
df['x2_bin'] = pd.cut(df['x_end'], bins = xT_cols, labels=False)
df['y2_bin'] = pd.cut(df['y_end'], bins = xT_rows, labels=False)
df['start_zone_value'] = df[['x1_bin','y1_bin']].apply(lambda x: xT[x[1]][x[0]],axis=1)
df['end_zone_value'] = df[['x2_bin','y2_bin']].apply(lambda x: xT[x[1]][x[0]],axis=1)
df['xT'] = df['end_zone_value'] - df['start_zone_value']

In [13]:
df = df.drop('Unnamed: 0',axis=1)

In [14]:
df.to_csv('passes.csv')